In [368]:
# importing pandas
import pandas as pd
# importing datetime
# importing geopandas
import geopandas as gpd
import datetime

### MASS SHOOTING (data cleaning)

In [369]:
# importing mass shooting data, finding shape
df = pd.read_csv('massshootingdata.csv')
df.shape

(2000, 11)

In [370]:
# suspects column with the number of suspects
df["suspects"]= (df["# Subjects-Suspects Injured"] 
                 + df["# Subjects-Suspects Killed"] 
                 + df["# Subjects-Suspects Arrested"])
# boolean column showing if a suspect was killed
df["suspectkilled"] = df["# Subjects-Suspects Killed"].map(lambda a : a > 0)

# dropping unnecessary columns
df = df.drop(["# Subjects-Suspects Injured", "# Subjects-Suspects Killed", 
              "# Subjects-Suspects Arrested", "Operations", "Address"], axis=1)


In [371]:
# renaming columns to clarify purpose & simplify the coding process
rename_dict = {
    "Incident ID": "id",
    "Incident Date": "date",
    "State": "state",
    "City Or County": "area",
    "# Victims Injured": "injuries",
    "# Victims Killed": "fatalities",
}
df.rename(columns=rename_dict, inplace=True)

In [372]:
df["tot_victims"] = df["injuries"] + df["fatalities"]

In [373]:
month_num = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# df["year"] = df.date.map(lambda a: int(a[-4:]))

def string_to_date(row):
    parts = row.date.split(" ")
    month = month_num[parts[0]]
    day = int(parts[1].rstrip(','))
    year = int(parts[2])
    row["date"] = datetime.date(year, month, day)
    row["year"] = year
    return row
    
df = df.apply(string_to_date, axis=1)

In [374]:
df = (df.loc[(df.year >= 2021) & (df.year <= 2022)]
      .reset_index()
      .drop("index", axis=1))

In [375]:
df.head()

,id,date,state,area,injuries,fatalities,suspects,suspectkilled,tot_victims,year
0,2492253,2022-12-31,Alabama,Mobile,7,1,2,False,8,2022
1,2491656,2022-12-31,Arizona,Phoenix,9,0,0,False,9,2022
2,2491054,2022-12-30,Tennessee,Memphis,3,1,0,False,4,2022
3,2490925,2022-12-30,Texas,Humble,3,2,0,False,5,2022
4,2489614,2022-12-28,New York,Bronx,3,1,2,False,4,2022


### GUN LAWS (data cleaning)

In [376]:
gun_laws_df = pd.read_csv("gunlaws.csv")

In [377]:
gun_laws_df = gun_laws_df.loc[gun_laws_df.year == 2020].drop("year", axis=1)

In [378]:
gun_laws_df.columns.values

array(['state', 'felony', 'invcommitment', 'invoutpatient', 'danger',
       'drugmisdemeanor', 'alctreatment', 'alcoholism', 'relinquishment',
       'violent', 'violenth', 'violentpartial', 'dealer', 'dealerh',
       'recordsall', 'recordsallh', 'recordsdealer', 'recordsdealerh',
       'reportall', 'reportallh', 'reportdealer', 'reportdealerh',
       'purge', 'residential', 'theft', 'security', 'inspection',
       'ammlicense', 'ammrecords', 'permit', 'permith', 'fingerprint',
       'training', 'permitlaw', 'registration', 'registrationh',
       'defactoreg', 'defactoregh', 'ammpermit', 'ammrestrict',
       'age21handgunsale', 'age18longgunsale', 'age21longgunsaled',
       'age21longgunsale', 'age21handgunpossess', 'age18longgunpossess',
       'age21longgunpossess', 'loststolen', 'amm18', 'amm21h',
       'universal', 'universalh', 'gunshow', 'gunshowh',
       'universalpermit', 'universalpermith', 'backgroundpurge',
       'ammbackground', 'threedaylimit', 'mentalhealth', 

In [379]:
gun_laws_df = gun_laws_df.set_index("state").sort_values(by='state')
gun_laws_df.head()

,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
state,,,,,,,,,,,,,,,,,,,,,
Alabama,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,10
Alaska,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
Arizona,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,8
Arkansas,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
California,1,1,0,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,111


### SHAPEFILES (data cleaning)

In [380]:
shape_df = gpd.read_file("shapefiles/states.shp")

In [381]:
territories = ["American Samoa", "Commonwealth of the Northern Mariana Islands", 
               "Puerto Rico", "United States Virgin Islands",
               "District of Columbia", "Guam"]
shape_df = (shape_df
            .loc[:, ["NAME", "STUSPS", "geometry"]]
            .rename(columns={"NAME": "state", "STUSPS": "abbreviation"})
            .set_index("state")
            .drop(territories, axis=0)
            .sort_values(by='state'))

In [382]:
shape_df.head()

,abbreviation,geometry
state,,
Alabama,AL,"POLYGON ((-9841333.855 3579579.495, -9841349.6..."
Alaska,AK,"MULTIPOLYGON (((-15108567.654 8339788.089, -15..."
Arizona,AZ,"POLYGON ((-12761162.105 4147165.875, -12761214..."
Arkansas,AR,"POLYGON ((-10515267.713 4101325.818, -10515269..."
California,CA,"MULTIPOLYGON (((-13060108.516 3854208.959, -13..."


In [383]:
shape_df.index

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object', name='state')

### POPULATION (data cleaning)

In [384]:
pop_df = pd.read_csv('populationdata.csv')

In [385]:
pop_df = (pop_df
            .loc[:, ["NAME", "POPESTIMATE2019"]]
            .rename(columns={"NAME": "state", "POPESTIMATE2019": "population"})
            .set_index("state"))

### BRINGING IT ALL TOGETHER!

In [388]:
states_df = shape_df.join(pop_df)

In [392]:
combo_df = pd.merge(df, states_df, on="state").sort_values(by="date", ascending=False)

In [393]:
combo_df

,id,date,state,area,injuries,fatalities,suspects,suspectkilled,tot_victims,year,abbreviation,geometry,population
0,2492253,2022-12-31,Alabama,Mobile,7,1,2,False,8,2022,AL,"POLYGON ((-9841333.855 3579579.495, -9841349.6...",4903185
39,2491656,2022-12-31,Arizona,Phoenix,9,0,0,False,9,2022,AZ,"POLYGON ((-12761162.105 4147165.875, -12761214...",7278717
90,2490925,2022-12-30,Texas,Humble,3,2,0,False,5,2022,TX,"POLYGON ((-11473106.144 3889876.760, -11473097...",28995881
58,2491054,2022-12-30,Tennessee,Memphis,3,1,0,False,4,2022,TN,"POLYGON ((-9945959.215 4370089.033, -9944979.2...",6829174
197,2489614,2022-12-28,New York,Bronx,3,1,2,False,4,2022,NY,"MULTIPOLYGON (((-8866092.533 5160809.769, -886...",19453561
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,1893754,2021-01-03,Florida,Miami,6,0,0,False,6,2021,FL,"MULTIPOLYGON (((-9734892.689 3569910.909, -973...",21477737
1302,1892114,2021-01-01,Iowa,Sioux City,4,1,3,False,5,2021,IA,"POLYGON ((-10312404.419 4952179.681, -10312552...",3155070
196,1892704,2021-01-01,Texas,Amarillo,3,1,1,False,4,2021,TX,"POLYGON ((-11473106.144 3889876.760, -11473097...",28995881
1171,1892034,2021-01-01,Arkansas,Fort Smith,6,0,2,False,6,2021,AR,"POLYGON ((-10515267.713 4101325.818, -10515269...",3017804
